<a href="https://colab.research.google.com/github/Reemasree/Sentiment-Analysis---Major-Project/blob/main/Major_Project_Lexiconbased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

** SENTIMENT ANALYSIS USING PYTHON- LEXICON BASED APPROACH (VADER SENTIMENT)**

In [ ]:
!pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs = SentimentIntensityAnalyzer()

In [ ]:
text = "The product is good except batter"
vs.polarity_scores(text)

{'compound': 0.4404, 'neg': 0.0, 'neu': 0.633, 'pos': 0.367}

In [ ]:
#  WEB SCRAPING
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

In [ ]:
urls = ['https://inshorts.com/en/read/politics','https://inshorts.com/en/read/business','https://inshorts.com/en/read/world']

def build_dataframe(urls):
  news_data = []
  for url in urls:
    news_category = url.split('/')[-1]   # to get the last category in the url
    data = requests.get(url)
    soup = BeautifulSoup(data.content)

    news_articles = [{'news_headline': headline.find('span',attrs={"itemprop":"headline"}).string,  # to know the headlin
                      'news_article': article.find('div',attrs={"itemprop":"articleBody"}).string, #to know the entire paragraph
                      'news_category': news_category}

                      for headline,article in zip(soup.find_all('div',class_=["news-card-title news-right-box"]),
                                                  soup.find_all('div',class_=["news-card-content news-right-box"]))]

    news_articles = news_articles[0:30]
    news_data.extend(news_articles)
  df=pd.DataFrame(news_data)
  df =df[['news_headline','news_article','news_category']]
  return df

In [ ]:
df = build_dataframe(urls)
df

,news_headline,news_article,news_category
0,Assam CM to contest polls from Majuli; Himanta...,The BJP on Friday released a list of 70 candid...,politics
1,BJP names ex-Bengal minister Suvendu as Nandig...,Former West Bengal minister and TMC leader Suv...,politics
2,TMC leader denied poll ticket; his supporters ...,Supporters of Trinamool Congress (TMC) leader ...,politics
3,Former TMC MP Dinesh Trivedi joins BJP ahead o...,"Former Railway Minister Dinesh Trivedi, who ha...",politics
4,"AIADMK allots 20 Tamil Nadu Assembly seats, Ka...","Tamil Nadu's ruling party, the All India Anna ...",politics
...,...,...,...
70,"Treasury yields signal recovery, not inflation...",US Treasury Secretary and former Fed chair Jan...,world
71,We don’t take advice from Trump on immigration...,Defending US President Joe Biden's administrat...,world
72,Being a royal is different than what people im...,"Britain's Duchess of Sussex, Meghan Markle, ha...",world
73,"Aus broadcaster suspends China's CGTN, CCTV ov...",Australia's public broadcaster SBS said it wou...,world


In [ ]:

#saving the dataset for future reference
df.to_csv('news.csv',index=False)

PREPROCESSING 


In [ ]:
# Dealing with punctuation and numbers
import re
def remove_sp(text):
  pattern = r'[^A-Za-z \s]'
  text = re.sub(pattern,'',text)
  return text


In [ ]:
# Tokenization and removing stop words
import nltk
nltk.download('stopwords')
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

def remove_stopwords(text):
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  filtered_tokens = [token for token in tokens if token not in stopword_list]
  filtered_text = ' '.join(filtered_tokens)

  return filtered_text


In [ ]:
df

,news_headline,news_article,news_category
0,Assam CM to contest polls from Majuli; Himanta...,The BJP on Friday released a list of 70 candid...,politics
1,BJP names ex-Bengal minister Suvendu as Nandig...,Former West Bengal minister and TMC leader Suv...,politics
2,TMC leader denied poll ticket; his supporters ...,Supporters of Trinamool Congress (TMC) leader ...,politics
3,Former TMC MP Dinesh Trivedi joins BJP ahead o...,"Former Railway Minister Dinesh Trivedi, who ha...",politics
4,"AIADMK allots 20 Tamil Nadu Assembly seats, Ka...","Tamil Nadu's ruling party, the All India Anna ...",politics
...,...,...,...
70,"Treasury yields signal recovery, not inflation...",US Treasury Secretary and former Fed chair Jan...,world
71,We don’t take advice from Trump on immigration...,Defending US President Joe Biden's administrat...,world
72,Being a royal is different than what people im...,"Britain's Duchess of Sussex, Meghan Markle, ha...",world
73,"Aus broadcaster suspends China's CGTN, CCTV ov...",Australia's public broadcaster SBS said it wou...,world


In [ ]:
#converting to lower case
df.news_article = df.news_article.apply(lambda x: x.lower())
df.news_headline = df.news_headline.apply(lambda x: x.lower())

#removing punctuation and numbers
df.news_article = df.news_article.apply(remove_sp)
df.news_headline = df.news_headline.apply(remove_sp)
print(df)
#tokenization
df.news_article = df.news_article.apply(remove_stopwords)
df.news_headline = df.news_headline.apply(remove_stopwords)
df

                                        news_headline  ... news_category
0   assam cm to contest polls from majuli himanta ...  ...      politics
1   bjp names exbengal minister suvendu as nandigr...  ...      politics
2   tmc leader denied poll ticket his supporters t...  ...      politics
3   former tmc mp dinesh trivedi joins bjp ahead o...  ...      politics
4   aiadmk allots  tamil nadu assembly seats kanya...  ...      politics
..                                                ...  ...           ...
70  treasury yields signal recovery not inflation ...  ...         world
71  we dont take advice from trump on immigration ...  ...         world
72  being a royal is different than what people im...  ...         world
73  aus broadcaster suspends chinas cgtn cctv over...  ...         world
74  direct attack on democracy us on chinas move t...  ...         world

[75 rows x 3 columns]


,news_headline,news_article,news_category
0,assam cm contest polls majuli himanta biswa fi...,bjp friday released list candidates upcoming t...,politics
1,bjp names exbengal minister suvendu nandigram ...,former west bengal minister tmc leader suvendu...,politics
2,tmc leader denied poll ticket supporters torch...,supporters trinamool congress tmc leader arabu...,politics
3,former tmc mp dinesh trivedi joins bjp ahead w...,former railway minister dinesh trivedi resigne...,politics
4,aiadmk allots tamil nadu assembly seats kanyak...,tamil nadus ruling party india anna dravida mu...,politics
...,...,...,...
70,treasury yields signal recovery not inflation ...,us treasury secretary former fed chair janet y...,world
71,dont take advice trump immigration policy whit...,defending us president joe bidens administrati...,world
72,royal different people imagine meghan,britains duchess sussex meghan markle said roy...,world
73,aus broadcaster suspends chinas cgtn cctv huma...,australias public broadcaster sbs said wouldnt...,world


In [ ]:
#stemming and lemmetization
from nltk.stem import PorterStemmer
ps = PorterStemmer()

def stemming(tokenized_text):
  text = [ps.stem(tokenized_text) for word in tokenized_text]
  new_text = ' '.join(text)
  return new_text
df


,news_headline,news_article,news_category
0,assam cm contest polls majuli himanta biswa fi...,bjp friday released list candidates upcoming t...,politics
1,bjp names exbengal minister suvendu nandigram ...,former west bengal minister tmc leader suvendu...,politics
2,tmc leader denied poll ticket supporters torch...,supporters trinamool congress tmc leader arabu...,politics
3,former tmc mp dinesh trivedi joins bjp ahead w...,former railway minister dinesh trivedi resigne...,politics
4,aiadmk allots tamil nadu assembly seats kanyak...,tamil nadus ruling party india anna dravida mu...,politics
...,...,...,...
70,treasury yields signal recovery not inflation ...,us treasury secretary former fed chair janet y...,world
71,dont take advice trump immigration policy whit...,defending us president joe bidens administrati...,world
72,royal different people imagine meghan,britains duchess sussex meghan markle said roy...,world
73,aus broadcaster suspends chinas cgtn cctv huma...,australias public broadcaster sbs said wouldnt...,world


In [ ]:
df.news_article = df.news_article.apply(stemming)
df.news_headline = df.news_headline.apply(stemming)
df.head()


,news_headline,news_article,news_category
0,assam cm contest polls majuli himanta biswa fi...,bjp friday released list candidates upcoming t...,politics
1,bjp names exbengal minister suvendu nandigram ...,former west bengal minister tmc leader suvendu...,politics
2,tmc leader denied poll ticket supporters torch...,supporters trinamool congress tmc leader arabu...,politics
3,former tmc mp dinesh trivedi joins bjp ahead w...,former railway minister dinesh trivedi resigne...,politics
4,aiadmk allots tamil nadu assembly seats kanyak...,tamil nadus ruling party india anna dravida mu...,politics


APPROACH TO SENTIMENT ANALYSIS

In [ ]:
df['compound'] = df['news_article'].apply(lambda x: vs.polarity_scores(x)['compound'])

In [ ]:
df
#postitive statement - compound > 0.05
#neutral statement - compoung > -0.05 and compound < 0.05
#negative statement - compound <= -0.05

,news_headline,news_article,news_category,compound
0,assam cm contest polls majuli himanta biswa fi...,bjp friday released list candidates upcoming t...,politics,1.0000
1,bjp names exbengal minister suvendu nandigram ...,former west bengal minister tmc leader suvendu...,politics,0.9915
2,tmc leader denied poll ticket supporters torch...,supporters trinamool congress tmc leader arabu...,politics,1.0000
3,former tmc mp dinesh trivedi joins bjp ahead w...,former railway minister dinesh trivedi resigne...,politics,1.0000
4,aiadmk allots tamil nadu assembly seats kanyak...,tamil nadus ruling party india anna dravida mu...,politics,1.0000
...,...,...,...,...
70,treasury yields signal recovery not inflation ...,us treasury secretary former fed chair janet y...,world,1.0000
71,dont take advice trump immigration policy whit...,defending us president joe bidens administrati...,world,0.9924
72,royal different people imagine meghan royal di...,britains duchess sussex meghan markle said roy...,world,-1.0000
73,aus broadcaster suspends chinas cgtn cctv huma...,australias public broadcaster sbs said wouldnt...,world,-1.0000


In [ ]:
df['Sentiment'] = df['compound'].apply(lambda x: 'Positive' if x >0.05  else ( 'Neutral' if x >-0.05 and x<0.05 else 'Negative'))

In [ ]:
df 

,news_headline,news_article,news_category,compound,Sentiment
0,assam cm contest polls majuli himanta biswa fi...,bjp friday released list candidates upcoming t...,politics,1.0000,Positive
1,bjp names exbengal minister suvendu nandigram ...,former west bengal minister tmc leader suvendu...,politics,0.9915,Positive
2,tmc leader denied poll ticket supporters torch...,supporters trinamool congress tmc leader arabu...,politics,1.0000,Positive
3,former tmc mp dinesh trivedi joins bjp ahead w...,former railway minister dinesh trivedi resigne...,politics,1.0000,Positive
4,aiadmk allots tamil nadu assembly seats kanyak...,tamil nadus ruling party india anna dravida mu...,politics,1.0000,Positive
...,...,...,...,...,...
70,treasury yields signal recovery not inflation ...,us treasury secretary former fed chair janet y...,world,1.0000,Positive
71,dont take advice trump immigration policy whit...,defending us president joe bidens administrati...,world,0.9924,Positive
72,royal different people imagine meghan royal di...,britains duchess sussex meghan markle said roy...,world,-1.0000,Negative
73,aus broadcaster suspends chinas cgtn cctv huma...,australias public broadcaster sbs said wouldnt...,world,-1.0000,Negative


In [ ]:
data = pd.DataFrame(df[['news_article', 'compound', 'Sentiment']].copy())



In [287]:
vs.polarity_scores("It is not good enough")

{'compound': -0.3412, 'neg': 0.376, 'neu': 0.624, 'pos': 0.0}